In [3]:
import sys
import os

print(sys.path)
sys.path.append("C:/Users/jz421/Desktop/GlobalLocal/IEEG_Pipelines/") #need to do this cuz otherwise ieeg isn't added to path...

# Get the absolute path to the directory containing the current script
# For GlobalLocal/src/analysis/preproc/make_epoched_data.py, this is GlobalLocal/src/analysis/preproc
try:
    # This will work if running as a .py script
    current_file_path = os.path.abspath(__file__)
    current_script_dir = os.path.dirname(current_file_path)
except NameError:
    # This will be executed if __file__ is not defined (e.g., in a Jupyter Notebook)
    # os.getcwd() often gives the directory of the notebook,
    # or the directory from which the Jupyter server was started.
    current_script_dir = os.getcwd()

# Navigate up three levels to get to the 'GlobalLocal' directory
project_root = os.path.abspath(os.path.join(current_script_dir, '..', '..', '..'))

# Add the 'GlobalLocal' directory to sys.path if it's not already there
if project_root not in sys.path:
    sys.path.insert(0, project_root) # insert at the beginning to prioritize it

import mne
import os
from ieeg.viz.mri import plot_subj, subject_to_info, plot_on_average #this used to import plot subj sig and nonsig, and plot electrodes from mri directly, but in order to save aaron's changes, i put them into jim_mri
from src.analysis.vis.jim_mri import plot_subj_sig_and_nonsig, _plot_electrodes, get_sub_dir
from ieeg.io import raw_from_layout, get_data
# from utils import load_sig_chans, get_sig_chans_per_subject, make_sig_electrodes_per_subject_and_roi_dict, channel_names_to_indices
from src.analysis.utils.general_utils import *

import json
import pyvista as pv
pv.global_theme.allow_empty_mesh = True

['C:\\Users\\jz421\\Desktop\\GlobalLocal\\IEEG_Pipelines', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\python311.zip', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\DLLs', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg', '', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\win32\\lib', 'C:\\Users\\jz421\\AppData\\Roaming\\Python\\Python311\\site-packages\\Pythonwin', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\win32\\lib', 'c:\\Users\\jz421\\AppData\\Local\\anaconda3\\envs\\ieeg\\Lib\\site-packages\\Pythonwin']


### plot single subject's chosen electrodes

In [ ]:
test = subject_to_info('D65')

In [ ]:
subj = "D0057"
base_path = r'C:\Users\jz421\Box\CoganLab\BIDS-1.1_GlobalLocal\BIDS\derivatives\freqFilt\figs'
plot_these_elecs = ['RAI2', 'RAI3', 'RAI4']
plot_these_elecs_save_name = '_'.join(plot_these_elecs)

subj_without_zeroes = "D" + subj[1:].lstrip('0')
brain_fig = plot_subj(subj_without_zeroes, picks=plot_these_elecs, labels_every=1, hemi='both', title=f'{subj}_Stimulus', background='black', color=(1, 1, 0)) 

# Specify the path and filename for the PNG file
png_filepath = os.path.join(base_path, f"{subj_without_zeroes}_{plot_these_elecs_save_name}.png")

# Save the figure
brain_fig.save_image(png_filepath, 'png')